### Load Libraries

In [11]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Input
from keras.layers import LSTM, SpatialDropout1D 
from sklearn.model_selection import train_test_split
import csv
import keras
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import os
#from getcpuinfo import cpuinfo
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import requests
from apscheduler.schedulers.background import BackgroundScheduler
import datetime
from keras import regularizers

### Load Raw Training Data

In [12]:
inpData = []
labels = []

'''
with open('cdata.csv') as csv_file:
    lines = csv.reader(csv_file, delimiter=',')
    for row in lines:
        labels.append(row[0])
        inpData.append(float(row[1]))
'''
inpDataArray = np.loadtxt("usageData.csv",delimiter = ',') 

### Set the Target Labels

In [13]:
minThreshold = 30
maxThreshold = 70
target = []
cpuUsage = []
historyParam = 50
for i in range(0,inpDataArray.shape[1]):
    inpData = inpDataArray[:,i]
    index = historyParam
    while index < len(inpData):
        currRec = inpData[index-historyParam:index]
        if inpData[index] > maxThreshold:# Scale Up
            target.append(1)
        elif inpData[index] < minThreshold: # Scale Down
            target.append(-1)
        else:
            target.append(0)  #Remain same
        cpuUsage.append(currRec)
        index += 1

In [14]:
cpuUsageData = np.expand_dims(np.array(cpuUsage,dtype = 'float'),axis = 2)
cpuScaling = np.array(target,dtype = 'float')
print(cpuScaling.shape)
print(cpuUsageData.shape)

cpuScalingCategorical = to_categorical(cpuScaling, num_classes = 3)

(47072,)
(47072, 50, 1)


### Preprocessing Input Data

In [15]:
'''cpuUsage = []
cpuScaling = []
historyParam = 50
index = historyParam

while index<len(inpData):
    currRec = [[inpData[i]] for i in range(index-historyParam,index)]
    cpuUsage.append(currRec)
    index+=1'''
    

'cpuUsage = []\ncpuScaling = []\nhistoryParam = 50\nindex = historyParam\n\nwhile index<len(inpData):\n    currRec = [[inpData[i]] for i in range(index-historyParam,index)]\n    cpuUsage.append(currRec)\n    index+=1'

In [16]:
'''cpuScaling = target[historyParam-1:len(target)]


cpuUsageData = np.array(cpuUsage,dtype = 'float')
cpuScaling = np.array(cpuScaling,dtype = 'float')

cpuScalingCategorical = to_categorical(cpuScaling, num_classes = 3)'''

"cpuScaling = target[historyParam-1:len(target)]\n\n\ncpuUsageData = np.array(cpuUsage,dtype = 'float')\ncpuScaling = np.array(cpuScaling,dtype = 'float')\n\ncpuScalingCategorical = to_categorical(cpuScaling, num_classes = 3)"

In [17]:
#len(inpData)
X_train, X_test, y_train, y_test = train_test_split(cpuUsageData , cpuScalingCategorical, test_size=0.10, random_state=3)

In [18]:
cpuUsageData.shape
#np.shape(cpuUsage)

(47072, 50, 1)

In [19]:
import pandas
testrec = 5
print(cpuScalingCategorical[testrec])
print(cpuScaling[testrec])
print(set(cpuScaling))
myseries=pandas.Series(cpuScaling)
print(myseries.value_counts())

[1. 0. 0.]
0.0
{0.0, 1.0, -1.0}
 0.0    20761
-1.0    13176
 1.0    13135
dtype: int64


In [20]:
model = Sequential()
#model.add(SpatialDropout1D(0.3))
model.add(LSTM((50),dropout = 0.3,batch_input_shape = (None,50,1), return_sequences = True))
model.add(Dense(25, activation = 'relu',
                    kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.1)))
model.add(LSTM(25,return_sequences = False))
model.add(Dense(12, activation = 'relu',
                    kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.1)))
#model.add(Dropout(rate = 0.3))
model.add(Dense(6, activation = 'relu',
                    kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.1)))
model.add(Dense(3, activation='softmax', kernel_initializer='he_normal'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam',metrics = ['accuracy'])
#model.summary()

In [21]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 50, 50)            10400     
_________________________________________________________________
dense_1 (Dense)              (None, 50, 25)            1275      
_________________________________________________________________
lstm_3 (LSTM)                (None, 25)                5100      
_________________________________________________________________
dense_2 (Dense)              (None, 12)                312       
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 78        
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 21        
Total params: 17,186
Trainable params: 17,186
Non-trainable params: 0
_________________________________________________________________


In [22]:
filepath="SimpleLSTMV3BestWeights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [23]:
'''if os.path.isfile("LSTMBestWeights.hdf5"):
    #print("Hey")
    model.load_weights("LSTMBestWeights.hdf5")'''

'if os.path.isfile("LSTMBestWeights.hdf5"):\n    #print("Hey")\n    model.load_weights("LSTMBestWeights.hdf5")'

In [24]:
history = model.fit(X_train, y_train, epochs = 25,validation_data = [X_test, y_test], callbacks = callbacks_list,verbose=1, batch_size = 1000)

Instructions for updating:
Use tf.cast instead.
Train on 42364 samples, validate on 4708 samples
Epoch 1/25
42364/42364 [==============================] - 13s 305us/step - loss: 8.2882 - acc: 0.3675 - val_loss: 7.1270 - val_acc: 0.5287

Epoch 00001: val_acc improved from -inf to 0.52867, saving model to SimpleLSTMV3BestWeights.hdf5
Epoch 2/25
42364/42364 [==============================] - 12s 279us/step - loss: 6.2073 - acc: 0.6517 - val_loss: 5.3106 - val_acc: 0.5847

Epoch 00002: val_acc improved from 0.52867 to 0.58475, saving model to SimpleLSTMV3BestWeights.hdf5
Epoch 3/25
42364/42364 [==============================] - 12s 281us/step - loss: 4.6415 - acc: 0.8097 - val_loss: 4.0564 - val_acc: 0.6017

Epoch 00003: val_acc improved from 0.58475 to 0.60174, saving model to SimpleLSTMV3BestWeights.hdf5
Epoch 4/25
42364/42364 [==============================] - 12s 276us/step - loss: 3.5565 - acc: 0.8390 - val_loss: 3.1742 - val_acc: 0.6226

Epoch 00004: val_acc improved from 0.60174 to 

In [ ]:
X_test.shape
np.expand_dims(X_test[0],axis = 0).shape

In [ ]:
res = model.predict(np.expand_dims(X_test[0],axis=0))

In [ ]:
print(res)
print(np.argmax(res[0]))

In [ ]:
np.argmax([0,1,0],axis = 0)

In [ ]:
np.argmax(res,axis = 1)

In [ ]:
print(inpData[0+1])